In [9]:
import cv2
import numpy as np
import face_recognition
import os
from datetime import datetime
from gtts import gTTS
import time

 
path = 'trainning images'#path of the images folder
images = []
classNames = []
myList = os.listdir(path)
for cl in myList:
    curImg = cv2.imread(f'{path}/{cl}')
    images.append(curImg)
    classNames.append(os.path.splitext(cl)[0])
    
    

def findEncodings(images):   #converting image BGR2RGb and into numpy array
    encodeList = []
    for img in images:
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        encode = face_recognition.face_encodings(img)[0]  
        encodeList.append(encode)
        #print(encodeList)
    return encodeList


 
def markAttendance(name):
    with open('Attendance.csv','r+') as f: #writing into csv file
        myDataList = f.readlines()
        nameList = []
        for line in myDataList:
            entry = line.split(',')
            nameList.append(entry[0])
            
                
        if name not in nameList:
            now = datetime.now()
            dtString1 = now.strftime('%H:%M:%S')
            dtString2 = now.strftime('%A')
            dtString3 = now.strftime('%d:%m:%Y')
            f.writelines(f'\n{name},{dtString1},{dtString2},{dtString3}')


            #it will speak the person name infront of camera using GTTS libaray
            language='en'
            output=gTTS(text=(f'hello {name}'),lang=language,slow=False)
            output.save("output.mp3")
            os.system("start output.mp3")
            time.sleep(3)
            os.remove('output.mp3')

            
            
encodeListKnown = findEncodings(images)

 
cap = cv2.VideoCapture(0)


 
while True:
    success, img = cap.read()
    #img = captureScreen()
    imgS = cv2.resize(img,(0,0),None,0.25,0.25)
    imgS = cv2.cvtColor(imgS, cv2.COLOR_BGR2RGB)
    
    #face_recogination libary detects the face by the help of measurenment of faces
 
    facesCurFrame = face_recognition.face_locations(imgS)
    encodesCurFrame = face_recognition.face_encodings(imgS,facesCurFrame)
    
    
 
    for encodeFace,faceLoc in zip(encodesCurFrame,facesCurFrame):
        matches = face_recognition.compare_faces(encodeListKnown,encodeFace)
        faceDis = face_recognition.face_distance(encodeListKnown,encodeFace)
        #print(faceDis)
        matchIndex = np.argmin(faceDis)
        
        
        
 
        if matches[matchIndex]:   #displaying persons name infront of camera
            name = classNames[matchIndex]
            #print(name)
            y1,x2,y2,x1 = faceLoc
            y1, x2, y2, x1 = y1*4,x2*4,y2*4,x1*4
            cv2.rectangle(img,(x1,y1),(x2,y2),(0,255,0),2)
            cv2.rectangle(img,(x1,y2-35),(x2,y2),(0,255,0),cv2.FILLED)
            cv2.putText(img,name,(x1+6,y2-6),cv2.FONT_HERSHEY_COMPLEX,1,(255,255,255),2)
            markAttendance(name)
            
            
            
 
    #cv2.imshow('Webcam',img)
    #cv2.waitKey(1)
    cv2.imshow('my video', img)
    if cv2.waitKey(13) & 0xFF == ord('q'):
        cv2.destroyAllWindows()
        break